In [1]:
import requests
import matplotlib.pyplot as plt
import json
import pandas as pd

In [2]:
endpoint = 'http://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD?format=json'
response = requests.get(endpoint)
res = response.json()
GDP_Data = pd.json_normalize(res[1]).head(5)

In [3]:
endpoint = 'http://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN?format=json'
response = requests.get(endpoint)
res = response.json()
Life_Expectancy = pd.json_normalize(res[1]).head(5)

In [4]:
GDP_Data.head(2)

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2022,3553.91337,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2017 internation...",ZH,Africa Eastern and Southern
1,AFE,2021,3519.17484,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2017 internation...",ZH,Africa Eastern and Southern


In [5]:
Life_Expectancy.head(2)

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2022,NaN,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
1,AFE,2021,62.45459,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern


In [6]:
GDP_Data.columns

Index(['countryiso3code', 'date', 'value', 'unit', 'obs_status', 'decimal',
       'indicator.id', 'indicator.value', 'country.id', 'country.value'],
      dtype='object')

In [7]:
GDP_Data = GDP_Data.drop(columns=['countryiso3code','unit', 'indicator.id', 'indicator.value','obs_status', 'decimal', 'country.id']).copy()

In [8]:
GDP_Data

,date,value,country.value
0,2022,3553.913370,Africa Eastern and Southern
1,2021,3519.174840,Africa Eastern and Southern
2,2020,3455.023119,Africa Eastern and Southern
3,2019,3648.220302,Africa Eastern and Southern
4,2018,3661.360566,Africa Eastern and Southern


In [9]:
Life_Expectancy.columns

Index(['countryiso3code', 'date', 'value', 'unit', 'obs_status', 'decimal',
       'indicator.id', 'indicator.value', 'country.id', 'country.value'],
      dtype='object')

In [10]:
Life_Expectancy = Life_Expectancy.drop(columns = ['countryiso3code', 'unit', 'obs_status', 'decimal',
       'indicator.id', 'indicator.value', 'country.id'])

In [11]:
Life_Expectancy

,date,value,country.value
0,2022,NaN,Africa Eastern and Southern
1,2021,62.454590,Africa Eastern and Southern
2,2020,63.313860,Africa Eastern and Southern
3,2019,63.755678,Africa Eastern and Southern
4,2018,63.365863,Africa Eastern and Southern


In [13]:
gdp_le = GDP_Data.merge(Life_Expectancy, on = 'country.value', how = 'inner')

In [14]:
gdp_le.head()

,date_x,value_x,country.value,date_y,value_y
0,2022,3553.91337,Africa Eastern and Southern,2022,NaN
1,2022,3553.91337,Africa Eastern and Southern,2021,62.454590
2,2022,3553.91337,Africa Eastern and Southern,2020,63.313860
3,2022,3553.91337,Africa Eastern and Southern,2019,63.755678
4,2022,3553.91337,Africa Eastern and Southern,2018,63.365863
